# 3. カイ２乗検定

## 3.1　チキンの売り上げは少ないのか

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap3/sec1.html
- テーマ
   1. 行列の作成
    - 二次元の行列を手作成する
    - 行/列にそれぞれラベルを付ける
    - 縦方向、横方向の合計値を計算し、データフレームに連結する
        - 追加する要素にもラベルを付ける(行、列)
   2. 行列の計算
    - この計算により期待度数を求める
- その他
    - テーマ1、2でセルを分けて実行する

In [34]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import math

# テーマ1 : 行列の作成
## 観測度数用データフレーム作成

# 行、列のラベル指定
row_list = ['wakwak', 'mogmog']
column_list = ['potato', 'chicken']

# 手書きでデータを入力
freq_observed = pd.DataFrame({column_list[0]: [435, 265],
                              column_list[1]: [165, 135]},
                              index=row_list)

# sumも加えた一覧データ(雛形)
freq_observed_all = freq_observed

# 横方向の合計値を算出＆名前付け＆DataFrameに連結(右側)
sum_store         = pd.DataFrame(freq_observed_all.sum(axis=1), columns = ['sum_store'])
freq_observed_all = pd.concat([freq_observed_all, sum_store], axis=1)

# 縦方向の合計値を算出＆名前付け＆DataFrameに連結(下側)
sum_product       = pd.DataFrame(freq_observed_all.sum(axis=0), columns = ['sum_product']).T
freq_observed_all = pd.concat([freq_observed_all, sum_product], axis=0)

# 表示
freq_observed_all
#freq_observed

,chicken,potato,sum_store
sum_product,300,700,1000


In [2]:
# テーマ2 : 行列の計算
## 期待度数用データフレーム作成

# sum_storeとsum_pdoductを乗算
freq_expected = (sum_store.dot(sum_product.drop('sum_store', axis=1).values)/1000)

# 列名付け直し
freq_expected.columns = sum_product.drop('sum_store', axis=1).columns.values

# sumも加えた一覧データ(雛形)
freq_expected_all = freq_expected

# sum_store, sum_productを連結
freq_expected_all = pd.concat([freq_expected_all, sum_store], axis=1)
freq_expected_all = pd.concat([freq_expected_all, sum_product], axis=0)

# 表示
freq_expected_all
#freq_expected

,chicken,potato,sum_store
wakwak,180.0,420.0,600
mogmog,120.0,280.0,400
sum_product,300.0,700.0,1000


## 3.2　カイ２乗値とカイ２乗分布

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap3/sec2.html
- テーマ
   1. 読み物オンリーページ
   
## 3.3　カイ２乗検定

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap3/sec3.html
- テーマ
   1. カイ2乗値を求める
    - 3.1章で作成した観測度数と期待度数から、カイ2乗値を算出する
    - 算出にはscipyの関数を用いる
   2. p値を求める
    - 算出したカイ2乗値から、現象が起こり得る確率を求める(累積分布関数を使用)
    - 参考URL : http://tanihito.hatenablog.com/entry/20101030/1288943347
- その他
    - scipy.stats.chi2の参考 : https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.stats.chi2.html

In [3]:
# カイ2乗値の算出：行単位(wakwak/mogmog)
chisquare_list = stats.chisquare(freq_observed, f_exp = freq_expected)

# 行単位で計算されたカイ2乗値を加算する
chi2_val = chisquare_list.statistic.sum()

# p値を算出する(自由度=(2-1)*(2-1)=1)
# cdf = 累積分布関数
df = 1
p_val = 1 - stats.chi2.cdf(chi2_val, df)

# 表示
print('chi2_val = ',chi2_val)
print('p_val    = ',p_val)

chi2_val =  4.46428571429
p_val    =  0.0346105575157


## 3.4　実践編：ハンバーガーを入れてみる

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap3/sec4.html
- テーマ
   1. 関数を作成する
    - 行列の期待度数作成用関数
    - 行列のカイ2乗値、p値を算出する関数
   2. 作成した関数の使用
    - 観測度数データから期待度数作成→カイ2乗値、p値を算出

In [48]:
# 観測度数元データ
freq_o_1 = pd.DataFrame({'potato'   : [435, 265],
                         'chicken'  : [165, 135],
                         'hamburger': [650, 350]},
                         index=['wakwak', 'mogmog'])

# 期待度数作成用関数
def calc_freq_expected(freq_o):
    sum_row = pd.DataFrame(freq_o.sum(axis=1), columns = ['sum_row'])
    sum_column = pd.DataFrame(freq_o.sum(axis=0), columns = ['sum_column']).T
    sum_val       = sum_row.sum()
    
    freq_e = (sum_row.dot(sum_column.values)/sum_val.values)
    freq_e.columns = sum_column.columns.values   
    return freq_e

# カイ2乗値、p値算出用関数
def calc_chisquare(freq_o, freq_e):
    df = (freq_e.shape[0] - 1) * (freq_e.shape[1] - 1)
    chisquare_list = stats.chisquare(freq_o, f_exp = freq_e)
    chi2_val = chisquare_list.statistic.sum()
    p_val = 1 - stats.chi2.cdf(chi2_val, df)
    return chi2_val, p_val

# 関数の使用
freq_e_1    = calc_freq_expected(freq_o_1)
chi2_list_1 = calc_chisquare(freq_o_1, freq_e_1)

# 表示
print('chi2_val = ', chi2_list_1[0])
print('p_val    = ', chi2_list_1[1])

chi2_val =  9.90476190476
p_val    =  0.00706656373146


## 3.9　通過テスト

- URL : http://kogolab.chillout.jp/elearn/hamburger/chap3/sec9.html
- テーマ
   1. 関数の使用
       - 1つ上で作成した関数を別のセルで実行する

In [46]:
# 観測度数元データ
freq_o_2 = pd.DataFrame({'sakura'   : [24, 8],
                         'momo'     : [18, 18]},
                         index=['kokugo', 'sansuu'])

# 関数の使用
freq_e_2    = calc_freq_expected(freq_o_2)
chi2_list_2 = calc_chisquare(freq_o_2, freq_e_2)

# 表示
print('chi2_val = ', chi2_list_2[0])
print('p_val    = ', chi2_list_2[1])

chi2_val =  4.48351648352
p_val    =  0.0342232366228
